In [11]:
import sys
import csv
import string
import re
import emoji
import nltk
#nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from spell_checker import SpellChecker

In [12]:
class Index:
    """
    This data structure is the value of the indices dictionary.
    """
    def __init__(self, size, pointer2postingsList):
        #size of the postings list
        
        self.size = size
        #pointer to the head of the postings list
        self.pointer2postingsList = pointer2postingsList

In [13]:
class PostingNode:
    """
    Linked list for the postings list
    """
    def __init__(self, val):
        self.val = val
        self.next = None

In [14]:
class SpellChecker(object):

    DEFAULT_DICTIONARIES = {'english': open('englishdic.sec', 'r').read().splitlines(),
            'german': open('germandic-utf8.sec', 'r').read().splitlines()}

    def __init__(self, lang_vocab: list, fdist: dict = None, max_edit_distance: int = 2):
        self.alphabet = 'aäbcdefghijklmnoöpqrsßtuüvwxyz'
        self.lang_vocab = {letter: [word.lower() for word in lang_vocab \
            if word.lower().startswith(letter)] for letter in self.alphabet}
        self.fdist = fdist
        self.max_edit_distance = max_edit_distance
        
        # If no fdist provided and dict is default English, use the Brown news corpus'.
        if self.fdist is None: 
            if lang_vocab == SpellChecker.DEFAULT_DICTIONARIES['english']:
                from nltk.corpus import brown
                from nltk import FreqDist

                self.fdist = FreqDist(w.lower() for w in brown.words(categories='news'))

            else:
                raise TypeError('No frequency distribution index provided.')

    def word_probability(self, word: str) -> int: 
        """Divides the frequency of a word by overall token count."""
        try:
            return self.fdist[word.lower()] / len(self.fdist.keys())
        except KeyError:
            return 0.0

    def spell_check(self, word: str) -> str:
        # return max(self.candidates(word), key=self.word_probability)
        cans = self.candidates(word)
        if cans: return list(cans)[0]

    def candidates(self, word: str) -> tuple:
        return (self.known([word.lower()]) or                       # word if it is known
                self.known(self.edit_distance1(word.lower())) or    # known words with edit distance 1
                self.known(self.edit_distance2(word.lower())) or    # known words with edit distance 2
                [word])                                     # word, unknown

    def known(self, words: list) -> set:
        return set(w for w in words if len(w) > 1 \
            and w.lower() in self.lang_vocab[w[0].lower()])

    def edit_distance1(self, word: str) -> set:
        """
        Thanks to Peter Norvig (http://norvig.com/spell-correct.html)

        Creates all the possible letter combinations that can be made
        with an edit distance of 1 to the word.

        splits = all ways of dividing the word, e.g. 
            'word' -> ('w', 'ord'); useful for making changes
        deletions = all ways of removing a single letter, e.g.
            'word'-> 'ord'
        transpositions = all ways of swapping two letters immediately
            adjacent to one another, e.g. 'word' -> 'owrd'
        replacements = all ways of replacing a letter with another
            letter, e.g. 'word' -> 'zord'
        insertions = all ways of inserting a letter at any point in the
            word, e.g. 'word' -> 'wgord'

        :param str word: the relevant word
        :return: a set of terms with an edit distance of 1 from the word
        :rtype: set
        """
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletions = [left + right[1:] for left, right in splits if right]
        transpositions = [left + right[1] + right[0] + right[2:]
                for left, right in splits if len(right) > 1]
        replacements = [left + letter + right[1:] for left, right
                in splits if right for letter in self.alphabet]
        insertions = [left + letter + right for left, right in splits
                for letter in self.alphabet]

        return set(deletions + transpositions + replacements + insertions)

    def edit_distance2(self, word: str) -> set:
        #TODO: Should be obsolete now
        """Simply runs edit_distance1 on every result from edit_distance1(word)"""
        return set(edit2 for edit in self.edit_distance1(word) for edit2
            in self.edit_distance1(edit))
        
    def edit_distanceN(self, word: str) -> set:
        #FIXME
        """Runs `edit_distance1` on the results of `edit_distance1` n times."""
        ret_val = set(word)

        for _ in range(self.max_edit_distance):
            for val in ret_val:
                ret_val = ret_val | self.edit_distance1(val)

        return list(ret_val)


In [16]:
class TwitterIR(object):
    """
    Main Class for the information retrieval task.
    """
    __slots__ = 'id2doc', 'tokenizer', 'unicodes2remove', 'indices', \
    'urlregex', 'punctuation', 'emojis', 'stop_words', 'englishdic', 'germandic', \
    'engSpellCheck', 'gerSpellCheck'

    def __init__(self):
        #the original mapping from the id's to the tweets, 
        #which is kept until the end to index the tweets
        self.id2doc = {}
        self.tokenizer = TweetTokenizer()
        #bunch of punctuation unicodes which are not in 'string.punctuation'
        self.unicodes2remove = [
            #all kinds of quotes
            u'\u2018', u'\u2019', u'\u201a', u'\u201b', u'\u201c',\
            u'\u201d', u'\u201e', u'\u201f', u'\u2014',
            #all kinds of hyphens
            u'\u002d', u'\u058a', u'\u05be', u'\u1400', u'\u1806',\
            u'\u2010', u'\u2011', u'\u2012', u'\u2013',
            u'\u2014', u'\u2015', u'\u2e17', u'\u2e1a', u'\u2e3a',\
            u'\u2e3b', u'\u2e40', u'\u301c', u'\u3030',
            u'\u30a0', u'\ufe31', u'\ufe32', u'\ufe58', u'\ufe63',\
            u'\uff0d', u'\u00b4'
        ]
        #the resulting datastructure which has the tokens as keys
        #and the Index objects as values
        self.indices = {}
        #regex to match urls (taken from the web)
        self.urlregex = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]'
                                   '|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        #keep @ to be able to recognize usernames
        self.punctuation = string.punctuation.replace('@', '') + \
        ''.join(self.unicodes2remove)
        self.punctuation = self.punctuation.replace('#', '')
        #a bunch of emoji unicodes
        self.emojis = ''.join(emoji.UNICODE_EMOJI)
        self.emojis = self.emojis.replace('#', '')
        #combined english and german stop words
        self.stop_words = set(stopwords.words('english') + stopwords.words('german'))
        self.englishdic = set()
        self.germandic = set()
        self.engSpellCheck = None
        self.gerSpellCheck = None
        

    def initId2doc(self, path):
        """
        Reads the file in and fills the id2doc datastructure.
        :param path: path to the tweets.csv file
        :return:
        """
        with open(path, 'r', encoding='utf-8', newline='') as f:
            r = csv.reader(f, delimiter='\t')
            for line in r:
                self.id2doc[line[1]] = line[4]
        f.close()
    
    def initLanguageDics(self, path_dir, filenameEN='englishdic.sec', filenameGER='germandic.sec'):
        if filenameEN:
            with open(path_dir+filenameEN, 'r', encoding='utf-8') as f:
                for word in f.readlines():
                    self.englishdic.add(word.lower().strip())
                f.close()
            
        if filenameGER:
            with open(path_dir+filenameGER, 'r', encoding='utf-8') as f:
                for word in f.readlines():
                    self.germandic.add(word.lower().strip())
                f.close()
                
    def _initSpellCheck(self, dic_path):
        return SpellChecker(open(dic_path).read().splitlines(),
                fdist={term: node.size for (term, node) in self.indices.items()}).spell_check



    def __len__(self):
        return len(self.indices.keys())
                

    def clean(self, s):
        """
        Normalizes a string (tweet) by removing the urls, punctuation, digits,
        emojis, by putting everything to lowercase and removing the
        stop words. Tokenization is performed aswell.
        
        :param s the string (tweet) to clean
        :return: returns a list of cleaned tokens
        """
        s = ' '.join(s.replace('[NEWLINE]', '').split())
        s = self.urlregex.sub('', s).strip()
        s = s.translate(str.maketrans('', '', self.punctuation + string.digits \
                                      + self.emojis)).strip()
        s = ' '.join(s.split())
        s = s.lower()
        s = self.tokenizer.tokenize(s)
        s = [w for w in s if w not in self.stop_words]
        return s

    def index(self, path):
        """
        1) call the method to read the file in
        2) iterate over the original datastructure id2doc which keeps the mapping
        of the tweet ids to the actual tweets and do:
            2a) preprocessing of the tweets
            2b) create a mapping from each token to its postings list (tokens2id)
        3) iterate over the just created mapping of tokens to their respective 
        postings lists (tokens2id) and do:
            3a) calculate the size of the postingslist
            3b) sort the postings list numerically in ascending order
            3c) create a linked list for the postings list
            3d) create the Index object with the size of the postings list and
            the pointer to the postings list - add to the resulting datastructure 
        :param path: the path to the tweets.csv file
        :return:
        """
        self.initLanguageDics('./', filenameGER='germandic-utf8.sec')
        self.initId2doc(path)
        self.engSpellCheck = self._initSpellCheck('englishdic.sec')
        self.gerSpellCheck = self._initSpellCheck('germandic-utf8.sec')
        tokens2id = {}
        for id,doc in self.id2doc.items():
            doc = self.clean(doc)
            language = self._detectLanguage(' '.join(doc))
            print(doc)
            print(language)

            for t in doc:
                print("Word to check: ",t)
                if language == 'english':
                    if t[0] not in ['@', '#'] and t not in self.englishdic:
                        print("EN Corrected from: ", t)
                        t = self.spellCheck(t, language)
                        print("To: ", t)
                elif language == 'german':
                    if t[0] not in ['@', '#'] and t not in self.germandic:
                        print("DE Corrected from: ", t)
                        t = self.spellCheck(t, language)
                        print("To: ", t)
                        
                if t in tokens2id.keys():
                    tokens2id[t].add(id)
                else:
                    #a set is used to avoid multiple entries of the same tweetID
                    tokens2id[t] = {id}

        for t,ids in tokens2id.items():
            #size of the postings list which belongs to token t
            size = len(ids)
            #sort in ascending order
            ids = sorted(ids)
            #use the first (and smallest) tweetID to be the head node of the 
            #linked list
            node = PostingNode(ids[0])
            #keep reference to the head of the linked list since node variable
            #is going to be overridden
            pointer = node
            for id in ids[1:]:
                #create further list items
                n = PostingNode(id)
                #and append to the linked list
                node.next = n
                #step further
                node = n
            #create the index object with size of the postings list 
            #and a link to the postings list itself
            i = Index(size, pointer)
            self.indices[t] = i
            
        
    def _detectLanguage(self, context):
        tokens = self.tokenizer.tokenize(context)
        stopsEN = [token for token in tokens if token in stopwords.words('english')]
        stopsDE = [token for token in tokens if token in stopwords.words('german')]
        if len(stopsEN) > len(stopsDE):
            return 'english'
        elif len(stopsDE) > len(stopsEN):
            return 'german'
        else:
            cleaned = self.clean(context)

            wordsEN = []
            wordsDE = []
            for token in cleaned:
                if token in self.englishdic:
                    wordsEN.append(token)
                if token in self.germandic:
                    wordsDE.append(token)
            if len(wordsEN) > len(wordsDE):
                return 'english'
            elif len(wordsDE) > len(wordsEN):
                return 'german'
            else:
                return 'english'

    def _query(self, term, lang):
        """
        Internal method to query for one term.
        :param: term the word which was queried for 
        :return: returns the Index object of the corresponding query term
        """
        print("old: ",term)
        if lang == 'english':
            if term not in self.englishdic:
                term = self.spellCheck(term, lang)
                print("new: ", term)
                print()
        elif lang == 'german':
            if term not in self.germandic:
                term = self.spellCheck(term, lang)
                print("new: ", term)
                print()
                
        try:
            return self.indices[term]
        except KeyError:
            return Index(0, PostingNode(''))
    
    def spellCheck(self, term, lang):
        return {'english': self.engSpellCheck,
                'german': self.gerSpellCheck}[lang](term)

    def query(self, *arg):
        """
        Query method which can take any number of terms as arguments.
        It uses the internal _query method to get the postings lists for the single 
        terms. It calculates the intersection of all postings lists.
        :param *arg term arguments
        :return: returns a list of tweetIDs which all contain the query terms
        """
        language = self._detectLanguage(' '.join([t for t in arg]))
        print(language)
        #[self._query(t, language) for t in arg if t not in self.stop_words]
        #sys.exit(0)
        
        #at this point it's a list of Index objects
        pointers = [self._query(t, language) for t in arg if t not in self.stop_words]
        #here the Index objects get sorted by the size of the 
        #postings list they point to
        pointers = sorted(pointers, key=lambda i: i.size)
        #here it becomes a list of pointers to the postings lists
        pointers = [i.pointer2postingsList for i in pointers]
        #first pointer
        intersection = pointers[0]
        #step through the pointers
        for p in pointers[1:]:
            #intersection between the new postings list and the so far
            #computed intersection
            intersection = self.intersect(intersection, p)
            #if at any point the intersection is empty there is 
            #no need to continue
            if not intersection:
                return []
        #convert the resulting intersection to a normal list
        rval = []
        pointer = intersection
        while pointer:
            rval.append(pointer.val)
            pointer = pointer.next
            
        return rval
    
    def intersect(self, pointer1, pointer2):
        """
        Computes the intersection for two postings lists.
        :param pointer1: first postings list
        :param pointer2: second postings list
        :return: returns the intersection 
        """
        #create temporary head node
        node = PostingNode('tmp')
        #keep reference to head node
        rvalpointer = node
        while pointer1 and pointer2:
            val1 = pointer1.val
            val2 = pointer2.val
            #only append to the linked list if the values are equal
            if val1 == val2:
                n = PostingNode(val1)
                node.next = n
                node = n
                pointer1 = pointer1.next
                pointer2 = pointer2.next
            #otherwise the postings list with the smaller value 
            #at the current index moves one forward
            elif val1 > val2:
                pointer2 = pointer2.next
            elif val1 < val2:
                pointer1 = pointer1.next
        #return from the second element on since the first was the temporary one
        return rvalpointer.next



In [18]:
twitterIR = TwitterIR()
twitterIR.index('tweets.csv')

['@knakatani', '@chikonjugular', '@joofford', '@steveblogs', 'says', 'lifetime', 'risk', 'cervical', 'cancer', 'japan', 'means', 'hpv', 'endemic', 'japan', 'screening', 'working', 'well']
english
Word to check:  @knakatani
Word to check:  @chikonjugular
Word to check:  @joofford
Word to check:  @steveblogs
Word to check:  says
Word to check:  lifetime
Word to check:  risk
Word to check:  cervical
Word to check:  cancer
Word to check:  japan
Word to check:  means
Word to check:  hpv
EN Corrected from:  hpv


To:  hew
Word to check:  endemic
Word to check:  japan
Word to check:  screening
Word to check:  working
Word to check:  well
['@fischerkurt', 'lady', 'whats', 'tumor', '#kippcharts']
english
Word to check:  @fischerkurt
Word to check:  lady
Word to check:  whats
Word to check:  tumor
Word to check:  #kippcharts
['@kingsofmetal', 'diagnoseverdacht', 'nunmal', 'schwer', 'gerade', 'hausarzt', 'blutbild', 'meist', 'sehen', 'gerade', 'hormone', 'überprüft', 'erklärbare', 'gewichtseinlagerungen', 'ja', 'wasser', 'fett', 'kind', 'tumor']
german
Word to check:  @kingsofmetal
Word to check:  diagnoseverdacht
Word to check:  nunmal
DE Corrected from:  nunmal


To:  neunmal
Word to check:  schwer
Word to check:  gerade
Word to check:  hausarzt
Word to check:  blutbild
Word to check:  meist
Word to check:  sehen
Word to check:  gerade
Word to check:  hormone
Word to check:  überprüft
Word to check:  erklärbare
Word to check:  gewichtseinlagerungen
Word to check:  ja
Word to check:  wasser
Word to check:  fett
Word to check:  kind
Word to check:  tumor
['@germanletsplay', '@quentin', '@lopoopl', '@levanni', '@igeloe', '@annelle', 'glückwunsch']
german
Word to check:  @germanletsplay
Word to check:  @quentin
Word to check:  @lopoopl
Word to check:  @levanni
Word to check:  @igeloe
Word to check:  @annelle
Word to check:  glückwunsch
['interesting', 'pcr', 'rate', 'major', 'centers', 'authors', 'argue', 'treatment', 'compliance', 'major', 'centers', 'see', 'database', 'think', 'rather', 'due', 'earlier', 'detection', 'smaller', 'tumors', 'pcr', 'look', 'deeper', '#crcsm']
english
Word to check:  interesting
Word to check:  pcr
EN Corrected from: 

To:  rap
Word to check:  salem
Word to check:  aufgrund
Word to check:  tumors
Word to check:  eingeschläfert
['cancerfighting', 'nanorobots', 'programmed', 'seek', 'destroy', 'tumors', 'study', 'shows', 'first', 'applications', 'dna', 'origami', 'nanomedicine', 'nanoroboter', 'schrumpfen', 'tumore', '#medtech']
english
Word to check:  cancerfighting
EN Corrected from:  cancerfighting


To:  cancerfighting
Word to check:  nanorobots
EN Corrected from:  nanorobots


...

To:  reintherapieren
['#usa', '#upi', '#news', 'broadcast', '#emetnewspress', 'obesity', 'may', 'cause', 'sudden', 'cardiac', 'arrest', 'young', 'people', 'study', 'says', 'obesity', 'high', 'blood', 'pressure', 'may', 'play', 'much', 'greater', 'role', 'sudden', 'cardiac', 'arrest', 'among', 'young', 'people', 'previously', 'thought', 'ne']
english
Word to check:  #usa
Word to check:  #upi
Word to check:  #news
Word to check:  broadcast
Word to check:  #emetnewspress
Word to check:  obesity
Word to check:  may
Word to check:  cause
Word to check:  sudden
Word to check:  cardiac
Word to check:  arrest
Word to check:  young
Word to check:  people
Word to check:  study
Word to check:  says
Word to check:  obesity


Word to check:  high
Word to check:  blood
Word to check:  pressure
Word to check:  may
Word to check:  play
Word to check:  much
Word to check:  greater
Word to check:  role
Word to check:  sudden
Word to check:  cardiac
Word to check:  arrest
Word to check:  among
Word to check:  young
Word to check:  people
Word to check:  previously
Word to check:  thought
Word to check:  ne
['leseempfehlung', 'extraordinary', 'correlation', 'obesity', 'social', 'inequality']
english
Word to check:  leseempfehlung
EN Corrected from:  leseempfehlung


To:  leseempfehlung
Word to check:  extraordinary
Word to check:  correlation
Word to check:  obesity
Word to check:  social
Word to check:  inequality
['@fazefuzzface', 'welcome', 'obesity']
english
Word to check:  @fazefuzzface
Word to check:  welcome
Word to check:  obesity
['@isonnylucas', 'thats', 'exactly', 'point', 'whataboutism', 'dont', 'want', 'face', 'problem', 'point', 'worse', 'problem', 'bfollowing', 'logic', 'yes', 'opioid', 'crisis', 'bad', 'obesity', 'affects', 'way', 'children', 'many', 'deathslike', 'never', 'solve', 'problem']
english
Word to check:  @isonnylucas
Word to check:  thats
Word to check:  exactly
Word to check:  point
Word to check:  whataboutism
EN Corrected from:  whataboutism


To:  whatabouts
Word to check:  dont
Word to check:  want
Word to check:  face
Word to check:  problem
Word to check:  point
Word to check:  worse
Word to check:  problem
Word to check:  bfollowing
EN Corrected from:  bfollowing
To:  following
Word to check:  logic
Word to check:  yes
Word to check:  opioid
EN Corrected from:  opioid
To:  apioid
Word to check:  crisis
Word to check:  bad
Word to check:  obesity
Word to check:  affects
EN Corrected from:  affects


To:  affect
Word to check:  way
Word to check:  children
EN Corrected from:  children
